In [1]:
using RigidBodyDynamics
using RigidBodyTreeInspector
using Interact

In [2]:
# Open the viewer application
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()

true

In [3]:
# Construct a mechanism
mechanism = rand_chain_mechanism(Float64, [QuaternionFloating{Float64}; [Revolute{Float64} for i = 1:5]]...)

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2
      Vertex: body3, Edge: joint3
        Vertex: body4, Edge: joint4
          Vertex: body5, Edge: joint5
            Vertex: body6, Edge: joint6

In [4]:
# Construct the visualizer interface, and get the :robot1 subtree
vis = DrakeVisualizer.Visualizer()[:robot1]

Visualizer with path prefix Symbol[:robot1] using LCM LCMCore.LCM(Ptr{Void} @0x0000000003cd5920,RawFD(57),LCMCore.Subscription[LCMCore.Subscription{LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}}(LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#7{DrakeVisualizer.CoreVisualizer}}(DrakeVisualizer.Comms.CommsT,DrakeVisualizer.handle_msg),Ptr{Void} @0x0000000003d19dd0)])

In [5]:
# Load the mechanism's geometry into the visualizer
RigidBodyTreeInspector.setgeometry!(vis, RigidBodyTreeInspector.create_geometry(mechanism))

# We can draw the mechanism at a single state:
state = MechanismState(Float64, mechanism)
RigidBodyTreeInspector.settransform!(vis, state)

true

In [6]:
# Or we can interactively explore the degrees of freedom of the mechanism.
# Note that the quaternion floating base joint, "joint1", has been given 
# six sliders. The first three correspond to the exponential map representation
# of its rotation, and the next three correspond to its translation. 
RigidBodyTreeInspector.inspect(mechanism; show_inertias=true, randomize_colors=true);

Interact.Options{:SelectionSlider,Float64}(Signal{Float64}(0.0, nactions=1),"joint1.1",0.0,"0.0",Interact.OptionDict(DataStructures.OrderedDict("-3.141592653589793"=>-3.14159,"-3.015928947446201"=>-3.01593,"-2.8902652413026093"=>-2.89027,"-2.7646015351590183"=>-2.7646,"-2.6389378290154264"=>-2.63894,"-2.5132741228718345"=>-2.51327,"-2.3876104167282426"=>-2.38761,"-2.261946710584651"=>-2.26195,"-2.1362830044410597"=>-2.13628,"-2.0106192982974673"=>-2.01062…),Dict(2.13628=>"2.1362830044410597",-3.01593=>"-3.015928947446201",-2.13628=>"-2.1362830044410597",1.88496=>"1.8849555921538756",1.75929=>"1.7592918860102842",2.63894=>"2.6389378290154264",-0.376991=>"-0.3769911184307753",0.251327=>"0.25132741228718347",-2.51327=>"-2.5132741228718345",1.25664=>"1.2566370614359172"…)),Any[],Any[],true,"horizontal")

Interact.Options{:SelectionSlider,Float64}(Signal{Float64}(0.0, nactions=1),"joint1.2",0.0,"0.0",Interact.OptionDict(DataStructures.OrderedDict("-3.141592653589793"=>-3.14159,"-3.015928947446201"=>-3.01593,"-2.8902652413026093"=>-2.89027,"-2.7646015351590183"=>-2.7646,"-2.6389378290154264"=>-2.63894,"-2.5132741228718345"=>-2.51327,"-2.3876104167282426"=>-2.38761,"-2.261946710584651"=>-2.26195,"-2.1362830044410597"=>-2.13628,"-2.0106192982974673"=>-2.01062…),Dict(2.13628=>"2.1362830044410597",-3.01593=>"-3.015928947446201",-2.13628=>"-2.1362830044410597",1.88496=>"1.8849555921538756",1.75929=>"1.7592918860102842",2.63894=>"2.6389378290154264",-0.376991=>"-0.3769911184307753",0.251327=>"0.25132741228718347",-2.51327=>"-2.5132741228718345",1.25664=>"1.2566370614359172"…)),Any[],Any[],true,"horizontal")

Interact.Options{:SelectionSlider,Float64}(Signal{Float64}(0.0, nactions=1),"joint1.3",0.0,"0.0",Interact.OptionDict(DataStructures.OrderedDict("-3.141592653589793"=>-3.14159,"-3.015928947446201"=>-3.01593,"-2.8902652413026093"=>-2.89027,"-2.7646015351590183"=>-2.7646,"-2.6389378290154264"=>-2.63894,"-2.5132741228718345"=>-2.51327,"-2.3876104167282426"=>-2.38761,"-2.261946710584651"=>-2.26195,"-2.1362830044410597"=>-2.13628,"-2.0106192982974673"=>-2.01062…),Dict(2.13628=>"2.1362830044410597",-3.01593=>"-3.015928947446201",-2.13628=>"-2.1362830044410597",1.88496=>"1.8849555921538756",1.75929=>"1.7592918860102842",2.63894=>"2.6389378290154264",-0.376991=>"-0.3769911184307753",0.251327=>"0.25132741228718347",-2.51327=>"-2.5132741228718345",1.25664=>"1.2566370614359172"…)),Any[],Any[],true,"horizontal")

Interact.Options{:SelectionSlider,Float64}(Signal{Float64}(0.0, nactions=1),"joint1.4",0.0,"0.0",Interact.OptionDict(DataStructures.OrderedDict("-3.141592653589793"=>-3.14159,"-3.015928947446201"=>-3.01593,"-2.8902652413026093"=>-2.89027,"-2.7646015351590183"=>-2.7646,"-2.6389378290154264"=>-2.63894,"-2.5132741228718345"=>-2.51327,"-2.3876104167282426"=>-2.38761,"-2.261946710584651"=>-2.26195,"-2.1362830044410597"=>-2.13628,"-2.0106192982974673"=>-2.01062…),Dict(2.13628=>"2.1362830044410597",-3.01593=>"-3.015928947446201",-2.13628=>"-2.1362830044410597",1.88496=>"1.8849555921538756",1.75929=>"1.7592918860102842",2.63894=>"2.6389378290154264",-0.376991=>"-0.3769911184307753",0.251327=>"0.25132741228718347",-2.51327=>"-2.5132741228718345",1.25664=>"1.2566370614359172"…)),Any[],Any[],true,"horizontal")

Interact.Options{:SelectionSlider,Float64}(Signal{Float64}(0.0, nactions=1),"joint1.5",0.0,"0.0",Interact.OptionDict(DataStructures.OrderedDict("-3.141592653589793"=>-3.14159,"-3.015928947446201"=>-3.01593,"-2.8902652413026093"=>-2.89027,"-2.7646015351590183"=>-2.7646,"-2.6389378290154264"=>-2.63894,"-2.5132741228718345"=>-2.51327,"-2.3876104167282426"=>-2.38761,"-2.261946710584651"=>-2.26195,"-2.1362830044410597"=>-2.13628,"-2.0106192982974673"=>-2.01062…),Dict(2.13628=>"2.1362830044410597",-3.01593=>"-3.015928947446201",-2.13628=>"-2.1362830044410597",1.88496=>"1.8849555921538756",1.75929=>"1.7592918860102842",2.63894=>"2.6389378290154264",-0.376991=>"-0.3769911184307753",0.251327=>"0.25132741228718347",-2.51327=>"-2.5132741228718345",1.25664=>"1.2566370614359172"…)),Any[],Any[],true,"horizontal")

...

Interact.Options{:SelectionSlider,Float64}(Signal{Float64}(0.0, nactions=1),"joint2.1",0.0,"0.0",Interact.OptionDict(DataStructures.OrderedDict("-3.141592653589793"=>-3.14159,"-3.015928947446201"=>-3.01593,"-2.8902652413026093"=>-2.89027,"-2.7646015351590183"=>-2.7646,"-2.6389378290154264"=>-2.63894,"-2.5132741228718345"=>-2.51327,"-2.3876104167282426"=>-2.38761,"-2.261946710584651"=>-2.26195,"-2.1362830044410597"=>-2.13628,"-2.0106192982974673"=>-2.01062…),Dict(2.13628=>"2.1362830044410597",-3.01593=>"-3.015928947446201",-2.13628=>"-2.1362830044410597",1.88496=>"1.8849555921538756",1.75929=>"1.7592918860102842",2.63894=>"2.6389378290154264",-0.376991=>"-0.3769911184307753",0.251327=>"0.25132741228718347",-2.51327=>"-2.5132741228718345",1.25664=>"1.2566370614359172"…)),Any[],Any[],true,"horizontal")

Interact.Options{:SelectionSlider,Float64}(Signal{Float64}(0.0, nactions=1),"joint3.1",0.0,"0.0",Interact.OptionDict(DataStructures.OrderedDict("-3.141592653589793"=>-3.14159,"-3.015928947446201"=>-3.01593,"-2.8902652413026093"=>-2.89027,"-2.7646015351590183"=>-2.7646,"-2.6389378290154264"=>-2.63894,"-2.5132741228718345"=>-2.51327,"-2.3876104167282426"=>-2.38761,"-2.261946710584651"=>-2.26195,"-2.1362830044410597"=>-2.13628,"-2.0106192982974673"=>-2.01062…),Dict(2.13628=>"2.1362830044410597",-3.01593=>"-3.015928947446201",-2.13628=>"-2.1362830044410597",1.88496=>"1.8849555921538756",1.75929=>"1.7592918860102842",2.63894=>"2.6389378290154264",-0.376991=>"-0.3769911184307753",0.251327=>"0.25132741228718347",-2.51327=>"-2.5132741228718345",1.25664=>"1.2566370614359172"…)),Any[],Any[],true,"horizontal")

Interact.Options{:SelectionSlider,Float64}(Signal{Float64}(0.0, nactions=1),"joint4.1",0.0,"0.0",Interact.OptionDict(DataStructures.OrderedDict("-3.141592653589793"=>-3.14159,"-3.015928947446201"=>-3.01593,"-2.8902652413026093"=>-2.89027,"-2.7646015351590183"=>-2.7646,"-2.6389378290154264"=>-2.63894,"-2.5132741228718345"=>-2.51327,"-2.3876104167282426"=>-2.38761,"-2.261946710584651"=>-2.26195,"-2.1362830044410597"=>-2.13628,"-2.0106192982974673"=>-2.01062…),Dict(2.13628=>"2.1362830044410597",-3.01593=>"-3.015928947446201",-2.13628=>"-2.1362830044410597",1.88496=>"1.8849555921538756",1.75929=>"1.7592918860102842",2.63894=>"2.6389378290154264",-0.376991=>"-0.3769911184307753",0.251327=>"0.25132741228718347",-2.51327=>"-2.5132741228718345",1.25664=>"1.2566370614359172"…)),Any[],Any[],true,"horizontal")

Interact.Options{:SelectionSlider,Float64}(Signal{Float64}(0.0, nactions=1),"joint5.1",0.0,"0.0",Interact.OptionDict(DataStructures.OrderedDict("-3.141592653589793"=>-3.14159,"-3.015928947446201"=>-3.01593,"-2.8902652413026093"=>-2.89027,"-2.7646015351590183"=>-2.7646,"-2.6389378290154264"=>-2.63894,"-2.5132741228718345"=>-2.51327,"-2.3876104167282426"=>-2.38761,"-2.261946710584651"=>-2.26195,"-2.1362830044410597"=>-2.13628,"-2.0106192982974673"=>-2.01062…),Dict(2.13628=>"2.1362830044410597",-3.01593=>"-3.015928947446201",-2.13628=>"-2.1362830044410597",1.88496=>"1.8849555921538756",1.75929=>"1.7592918860102842",2.63894=>"2.6389378290154264",-0.376991=>"-0.3769911184307753",0.251327=>"0.25132741228718347",-2.51327=>"-2.5132741228718345",1.25664=>"1.2566370614359172"…)),Any[],Any[],true,"horizontal")

Interact.Options{:SelectionSlider,Float64}(Signal{Float64}(0.0, nactions=1),"joint6.1",0.0,"0.0",Interact.OptionDict(DataStructures.OrderedDict("-3.141592653589793"=>-3.14159,"-3.015928947446201"=>-3.01593,"-2.8902652413026093"=>-2.89027,"-2.7646015351590183"=>-2.7646,"-2.6389378290154264"=>-2.63894,"-2.5132741228718345"=>-2.51327,"-2.3876104167282426"=>-2.38761,"-2.261946710584651"=>-2.26195,"-2.1362830044410597"=>-2.13628,"-2.0106192982974673"=>-2.01062…),Dict(2.13628=>"2.1362830044410597",-3.01593=>"-3.015928947446201",-2.13628=>"-2.1362830044410597",1.88496=>"1.8849555921538756",1.75929=>"1.7592918860102842",2.63894=>"2.6389378290154264",-0.376991=>"-0.3769911184307753",0.251327=>"0.25132741228718347",-2.51327=>"-2.5132741228718345",1.25664=>"1.2566370614359172"…)),Any[],Any[],true,"horizontal")

In [7]:
# We can also animate the mechanism, given a time sequence of states.
# States will be linearly interpolated between the knot points.
times = collect(linspace(0, 10, 3))
configurations = Vector{Vector{Float64}}(length(times))
for i = 1:length(times)
    rand_configuration!(state)
    configurations[i] = copy(configuration_vector(state))
end
setgeometry!(vis, create_geometry(mechanism))
animate(vis, mechanism, times, configurations; fps = 60., realtimerate = 1.)

In [8]:
# We can simulate a mechanism from an initial state. We'll use a mechanism
# without a QuaternionFloating joint so that it doesn't just fall straight down.
mechanism = rand_chain_mechanism(Float64, [Revolute{Float64} for i = 1:10]...)
vis = Visualizer()[:robot2]
setgeometry!(vis, create_geometry(mechanism))
state = MechanismState(Float64, mechanism)
settransform!(vis, state)
times, configurations, velocities = simulate(state, 10.);

In [9]:
# And we can animate the result in realtime
animate(vis, mechanism, times, configurations)

In [10]:
# We can also inspect individual frames from the simulation
@manipulate for i in 1:length(times)
    set_configuration!(state, configurations[i])
    set_velocity!(state, velocities[i])
    settransform!(vis, state)
    times[i]
end

Interact.Options{:SelectionSlider,Int64}(Signal{Int64}(50001, nactions=1),"i",50001,"50001",Interact.OptionDict(DataStructures.OrderedDict("1"=>1,"2"=>2,"3"=>3,"4"=>4,"5"=>5,"6"=>6,"7"=>7,"8"=>8,"9"=>9,"10"=>10…),Dict(62417=>"62417",62682=>"62682",62591=>"62591",43031=>"43031",22035=>"22035",39001=>"39001",61670=>"61670",50996=>"50996",48586=>"48586",77654=>"77654"…)),Any[],Any[],true,"horizontal")

5.000000000001686